## Using Segmenting and KNN to decide optimal neighbourhoods in which to open a coffee shop in Toronto, CA

#### This notebook will use the FourSquare API along with downloaded data sets in order to cluster neighbourhoods in Toronto based on criteria in order to find an optimal place to open a coffee shop

This will happen through the creation of a pandas dataframe containing neighbourhood information, calls to the FourSquare API in order to identify venues within Toronto, and the use of distance matrixes and correlation between venues in order to find ten neighbourhoods which may be optimal to open a coffee shop in

First import all the libraries and dependencies needed

In [1]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np # library to handle data in a vectorized manner

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')
print('Libraries imported.')

Libraries imported.
Libraries imported.


Then store the web address in a variable for easy use

In [2]:
data_html = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

Using pandas.read_html read in the data from the website, taking in only the useful table which contains the information for this project. Then check what the first ten entries look like

In [3]:
neighbourhoods = pd.read_html(data_html, header=0)[0]
neighbourhoods.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


Drop all the entries in the table where the borough is Not assigned, and rename the post code column to postal code

In [4]:
neighbourhoods.columns=['Postal Code', 'Borough', 'Neighbourhood']
neighbourhoods = neighbourhoods[neighbourhoods.Borough!='Not assigned']
neighbourhoods.head(10)

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


Next step through the dataframe, find the entries/instances which have a not-assigned neighbourhood and storing them in a list. Then step through the list, replace the entries with their borough as the neighbourhood and check the first ten entries to see if changes have taken place

In [5]:
n_list = []
for row in neighbourhoods.itertuples():
    if row.Neighbourhood == 'Not assigned':
        n_list.append([row.Neighbourhood, row.Borough])

length = len(n_list)
i = 0
while i < length:
    borough = n_list[i][1]
    neighbourhoods.loc[neighbourhoods['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = borough
    i = i+1
neighbourhoods.head(10)

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


Finally merge the entries with the same Postcode and Borough, merging their neighbourhoods together with commas and spaces for easy reading. Then drop the duplicates which are created through this method and check the first ten entries in the dataframe

In [6]:
neighbourhoods['Neighbourhood'] = neighbourhoods.groupby(['Postal Code','Borough'])['Neighbourhood'].transform(lambda x: ', '.join(x)) 
neighbourhoods.drop_duplicates(inplace=True)
neighbourhoods.head(10)

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Harbourfront, Regent Park"
6,M6A,North York,"Lawrence Heights, Lawrence Manor"
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,"Rouge, Malvern"
14,M3B,North York,Don Mills North
15,M4B,East York,"Woodbine Gardens, Parkview Hill"
17,M5B,Downtown Toronto,"Ryerson, Garden District"


Finally print the shape of the dataframe (103,3)

In [7]:
neighbourhoods.shape

(103, 3)

Next, load in the Geospatial data from the CSV due to issues with reliability of Geocoder, then merge the two dataframes together and print the head

In [8]:
df = pd.read_csv('Geospatial_Coordinates.csv')
df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
full_data = pd.merge(neighbourhoods,df)
full_data.head(10)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


Geographical co-ordinates of Toronto stored in variables for later use:

In [10]:
address = 'Toronto Canada'
geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [11]:
# create map of Toronto using latitude and longitude values
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(full_data['Latitude'], full_data['Longitude'], full_data['Borough'], full_data['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

In [12]:
CLIENT_ID = '***' # your Foursquare ID
CLIENT_SECRET = '***' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT=100
radius = 500 # define radius

Use the get category type and get nearby venues functions from the lab

In [13]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Run the code above from the labs and create a dataframe of toronto venues

In [15]:
toronto_venues = getNearbyVenues(names=full_data['Neighbourhood'],
                                   latitudes=full_data['Latitude'],
                                   longitudes=full_data['Longitude']
                                  )

print(toronto_venues.shape)
print('There are {} uniques categories of neighbourhood.'.format(len(toronto_venues['Neighbourhood'].unique())))
toronto_venues.head()


Parkwoods
Victoria Village
Harbourfront, Regent Park
Lawrence Heights, Lawrence Manor
Queen's Park
Islington Avenue
Rouge, Malvern
Don Mills North
Woodbine Gardens, Parkview Hill
Ryerson, Garden District
Glencairn
Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park
Highland Creek, Rouge Hill, Port Union
Flemingdon Park, Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Downsview North, Wilson Heights
Thorncliffe Park
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
East Birchmount Park, Ionview, Kennedy Park
Bayview Village
CFB Toronto, Downsview East
The D

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


Then check how many venues were given per neighbourhood

In [16]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,5,5,5,5,5,5
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",11,11,11,11,11,11
"Alderwood, Long Branch",9,9,9,9,9,9
"Bathurst Manor, Downsview North, Wilson Heights",17,17,17,17,17,17
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",25,25,25,25,25,25
Berczy Park,56,56,56,56,56,56


Next, begin to analyse the neighbourhoods by one hot encoding

In [17]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head(10)

(2245, 275)


,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

Next, merge the columns of Coffee shop, cafe and gaming cafe due to their similarities

In [18]:
toronto_onehot['Coffee Shop'] = toronto_onehot['Coffee Shop'] + toronto_onehot['Café'] + toronto_onehot['Gaming Cafe']
del toronto_onehot['Café']
del toronto_onehot['Gaming Cafe']
toronto_onehot.head()

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

Group the rows by neighbourhood and take the mean frequency of occurence of each category

In [19]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
print(toronto_grouped.shape)
toronto_grouped.head()

(99, 273)


,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03,0.0,0.0,0.0,0.01,0.01,0.0,0.03,0.0,0.0,0.0,0.0

Check the correlation of the venues, pulling out the list of venue correlations to the coffee shops in order to get a statistical idea of how neighbourhoods are structured

In [20]:
toronto_correlation = toronto_grouped.corr()
toronto_correlation
toronto_correlation.sort_values(by = 'Coffee Shop', ascending = False, inplace = True)
coffee_correlation = toronto_correlation['Coffee Shop']
np.round(coffee_correlation, 3)


Coffee Shop                        1.000
Korean Restaurant                  0.560
Nightclub                          0.256
Restaurant                         0.246
Diner                              0.235
Theater                            0.230
French Restaurant                  0.212
Seafood Restaurant                 0.211
Salad Place                        0.208
Metro Station                      0.185
Vegetarian / Vegan Restaurant      0.183
Art Gallery                        0.178
Portuguese Restaurant              0.176
College Auditorium                 0.171
Wine Bar                           0.170
Baby Store                         0.170
Office                             0.167
Hotel Bar                          0.166
Japanese Restaurant                0.165
Neighborhood                       0.165
Creperie                           0.164
Health Food Store                  0.164
Performing Arts Venue              0.164
Pub                                0.164
History Museum  

In order to inspect the correlation more closely, take the top ten venues which correlate to coffee shops and the ten which least correlate

In [21]:
coffee_correlation.head(11)

Coffee Shop                      1.000000
Korean Restaurant                0.560433
Nightclub                        0.256429
Restaurant                       0.246443
Diner                            0.234836
Theater                          0.229822
French Restaurant                0.212071
Seafood Restaurant               0.210903
Salad Place                      0.208110
Metro Station                    0.185370
Vegetarian / Vegan Restaurant    0.182546
Name: Coffee Shop, dtype: float64

In [22]:
coffee_correlation.tail(10)

Fast Food Restaurant         -0.118299
Construction & Landscaping   -0.120150
Baseball Field               -0.120438
Shopping Mall                -0.120960
Athletics & Sports           -0.123702
Home Service                 -0.149449
Trail                        -0.152353
Playground                   -0.155235
Bus Line                     -0.198825
Park                         -0.269596
Name: Coffee Shop, dtype: float64

From this we can see that in the neighbourhoods where there are venues such as the Korean Restaurants and Nightclubs there are more likely to be coffee shops. As such these are neighbourhoods where opening a new coffee shop is less likely to succeed due to the high competition of coffee shops already being there. On the other end of the correlation, where there is a venue such as a park or playground there is less likely to be a coffee shop so it is worth looking into these neighbourhoods to see whether a coffee shop would be viable.

Now print the neighbourhood with their top 5 venues to visualise how the venues relate to each other

In [23]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    #print(temp)
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
             venue  freq
0      Coffee Shop  0.11
1       Steakhouse  0.04
2              Bar  0.04
3  Thai Restaurant  0.04
4     Burger Joint  0.03


----Agincourt----
                venue  freq
0      Clothing Store   0.2
1      Breakfast Spot   0.2
2  Chinese Restaurant   0.2
3              Lounge   0.2
4      Sandwich Place   0.2


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                venue  freq
0          Playground   0.5
1                Park   0.5
2   Accessories Store   0.0
3  Mexican Restaurant   0.0
4               Motel   0.0


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                 venue  freq
0        Grocery Store  0.18
1           Beer Store  0.09
2       Sandwich Place  0.09
3  Japanese Restaurant  0.09
4  Fried Chicken Joint  0.09


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.22
1     Coffee Shop  

4  Middle Eastern Restaurant   0.0


----East Birchmount Park, Ionview, Kennedy Park----
                venue  freq
0      Discount Store  0.29
1   Convenience Store  0.14
2         Bus Station  0.14
3  Chinese Restaurant  0.14
4         Coffee Shop  0.14


----East Toronto----
               venue  freq
0               Park  0.25
1  Convenience Store  0.25
2        Coffee Shop  0.25
3      Metro Station  0.25
4  Accessories Store  0.00


----Emery, Humberlea----
                       venue  freq
0             Baseball Field   1.0
1  Middle Eastern Restaurant   0.0
2              Movie Theater   0.0
3                      Motel   0.0
4        Monument / Landmark   0.0


----Fairview, Henry Farm, Oriole----
                  venue  freq
0        Clothing Store  0.11
1  Fast Food Restaurant  0.08
2           Coffee Shop  0.06
3     Electronics Store  0.03
4                Bakery  0.03


----First Canadian Place, Underground city----
         venue  freq
0  Coffee Shop  0.15
1        Ho

4  Italian Restaurant  0.05


----The Junction North, Runnymede----
                venue  freq
0         Pizza Place  0.25
1            Bus Line  0.25
2   Convenience Store  0.25
3       Grocery Store  0.25
4  Miscellaneous Shop  0.00


----The Kingsway, Montgomery Road, Old Mill North----
                       venue  freq
0                       Park  0.33
1                      River  0.33
2                       Pool  0.33
3          Accessories Store  0.00
4  Middle Eastern Restaurant  0.00


----Thorncliffe Park----
               venue  freq
0  Indian Restaurant  0.12
1     Sandwich Place  0.12
2        Yoga Studio  0.06
3        Pizza Place  0.06
4               Park  0.06


----Victoria Village----
                   venue  freq
0            Pizza Place  0.25
1           Hockey Arena  0.25
2            Coffee Shop  0.25
3  Portuguese Restaurant  0.25
4     Miscellaneous Shop  0.00


----Westmount----
                       venue  freq
0                Pizza Place  0.25
1     

Now store the information in pandas dataframe, borrowing the most common venues sorting function from the lab before displaying the top ten venues for each neighbourhood

In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()


,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Thai Restaurant,Bar,Steakhouse,Hotel,Restaurant,Burger Joint,Bakery,Sushi Restaurant,American Restaurant
1,Agincourt,Chinese Restaurant,Lounge,Sandwich Place,Clothing Store,Breakfast Spot,Yoga Studio,Dog Run,Doner Restaurant,Donut Shop,Drugstore
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Yoga Studio,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Fast Food Restaurant,Coffee Shop,Japanese Restaurant,Discount Store,Sandwich Place,Beer Store,Fried Chicken Joint,Pizza Place,Pharmacy
4,"Alderwood, Long Branch",Pizza Place,Gym,Coffee Shop,Skating Rink,Pharmacy,Sandwich Place,Pub,Pool,Yoga Studio,Discount Store


Next create a dataframe which stores all neighbourhoods which have coffee shop as their 1st or second most common venue for analysis, and to remove them later from being chosen as a potential neighbourhood. This is due to their common status within this neighbourhood suggesting oversaturation and a wide range of coffee shops already existing in the neighbourhood

In [25]:
top_three_venues = pd.DataFrame()
top_one = pd.DataFrame()
top_two = pd.DataFrame()
top_three = pd.DataFrame()

top_venues = neighbourhoods_venues_sorted.copy()


top_two_venues = top_venues[top_venues['1st Most Common Venue'] == 'Coffee Shop']
top_two_venues = top_two_venues.append(top_venues[top_venues['2nd Most Common Venue'] == 'Coffee Shop'])
top_two_venues

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Thai Restaurant,Bar,Steakhouse,Hotel,Restaurant,Burger Joint,Bakery,Sushi Restaurant,American Restaurant
5,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Sandwich Place,Supermarket,Restaurant,Deli / Bodega,Fried Chicken Joint,Frozen Yogurt Shop,Bank,Diner,Bridal Shop
7,"Bedford Park, Lawrence Manor East",Coffee Shop,Italian Restaurant,Fast Food Restaurant,Pharmacy,Sandwich Place,Restaurant,Butcher,Pub,Pizza Place,Comfort Food Restaurant
8,Berczy Park,Coffee Shop,Cocktail Bar,Steakhouse,Cheese Shop,Restaurant,Farmers Market,Italian Restaurant,Seafood Restaurant,Bakery,Beer Bar
11,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Breakfast Spot,Pet Store,Italian Restaurant,Stadium,Caribbean Restaurant,Burrito Place,Furniture / Home Store,Climbing Gym,Bar
15,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Pub,Bakery,Pizza Place,Italian Restaurant,Japanese Restaurant,Liquor Store,Deli / Bodega,Bank
17,Canada Post Gateway Processing Centre,Coffee Shop,Hotel,American Restaurant,Burrito Place,Sandwich Place,Middle Eastern Restaurant,Fried Chicken Joint,Mediterranean Restaurant,Gym / Fitness Center,Dumpling Restaurant
19,Central Bay Street,Coffee Shop,Italian Restaurant,Bubble Tea Shop,Burger Joint,Bar,Chinese Restaurant,Thai Restaurant,Salad Place,Spa,Sushi Restaurant
20,"Chinatown, Grange Park, Kensington Market",Coffee Shop,Bar,Vegetarian / Vegan Restaurant,Bakery,Chinese Restaurant,Mexican Restaurant,Arts & Crafts Store,Vietnamese Restaurant,Caribbean Restaurant,Comfort Food Restaurant
21,Christie,Coffee Shop,Grocery Store,Park,Nightclub,Convenience Store,Diner,Restaurant,Italian Restaurant,Baby Store,Eastern European Restaurant


Next we will create a dataframe which contains all other neighbourhoods, as they will be the search criteria to find a neighbourhood for the new coffee shop

In [26]:

top_tens = neighbourhoods_venues_sorted.copy()
potential_neighbourhoods = top_tens[top_tens['1st Most Common Venue'] != 'Coffee Shop']

indexNames = potential_neighbourhoods[potential_neighbourhoods['2nd Most Common Venue'] == 'Coffee Shop' ].index
potential_neighbourhoods.drop(indexNames, inplace=True)
print(potential_neighbourhoods.shape)
potential_neighbourhoods

(60, 11)


C:\Users\Ash\Anaconda3\lib\site-packages\pandas\core\frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Agincourt,Chinese Restaurant,Lounge,Sandwich Place,Clothing Store,Breakfast Spot,Yoga Studio,Dog Run,Doner Restaurant,Donut Shop,Drugstore
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Yoga Studio,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Fast Food Restaurant,Coffee Shop,Japanese Restaurant,Discount Store,Sandwich Place,Beer Store,Fried Chicken Joint,Pizza Place,Pharmacy
4,"Alderwood, Long Branch",Pizza Place,Gym,Coffee Shop,Skating Rink,Pharmacy,Sandwich Place,Pub,Pool,Yoga Studio,Discount Store
10,"Bloordale Gardens, Eringate, Markland Wood, Ol...",Liquor Store,Pharmacy,Pizza Place,Beer Store,Coffee Shop,Convenience Store,Shopping Plaza,Yoga Studio,Drugstore,Discount Store
12,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Garden Center,Smoke Shop,Farmers Market,Light Rail Station,Spa,Fast Food Restaurant,Brewery,Burrito Place,Restaurant
13,"CFB Toronto, Downsview East",Park,Airport,Bus Stop,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
14,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Terminal,Airport Service,Harbor / Marina,Boat or Ferry,Sculpture Garden,Plane,Boutique,Airport Gate,Airport
16,Caledonia-Fairbanks,Park,Fast Food Restaurant,Market,Pharmacy,Women's Store,Gym,Department Store,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
18,Cedarbrae,Athletics & Sports,Hakka Restaurant,Lounge,Fried Chicken Joint,Bakery,Caribbean Restaurant,Thai Restaurant,Bank,Donut Shop,Dog Run


From this we can see that there are 61 potential neighbourhoods in which to open the coffee shop, as they are not already oversaturated with coffee shops (i.e. do not have a coffee shop as their 1st or 2nd most common venue). From here we will need to identify a dummy neighbourhood which has information from the correlation and from the top ten venues lists in order to compare this dummy neighbourhood and find the ten closest to the dummy neighbourhood

First we need to find out a way to weight the correlation values to compare them to the mean frequency counts and get a dummy neighbourhood to compare

Once that has happened we will calculate the distance between all neighbourhoods and this new neighbourhood, organising the dataframe in order to remove all neighbourhoods which are also in the top already oversaturated neighbourhoods and to sort using KNN the five-ten most ideal neighbourhoods

In [27]:
#Recreate the coffee correlation where the values are in alphabetical order
toronto_correlation = toronto_grouped.corr()
dummy = toronto_correlation['Coffee Shop']

#Convert from the dataframe to a list for easy manipulation
n = len(dummy)
i = 0
d = []

while i < n:
    a = dummy[i]
    d.append(a)
    i = i + 1

#Now create the dummy neighbourhood that will be the base for our KNN
for idx in range(len(d)):
#First, change the top ten venues which all have correlation values over 0.2 from their correlation values to 0, as we do not want to place the coffee shop in a neighbourhood already overpopulated with coffee shops. only take the top ten as we are still attempting to
#keep in some desired neighbourhoods which may have coffee shops as their 3rd or less most common venues

    if d[idx] == 1.0:
        d[idx] = 0.2
    
    elif d[idx] > 0.2:
        d[idx] = 0.0

#Then, change the all correlation values which are greater than 0.1 to 0.1, reducing their impact on the final
#dummy neighbourhood value. Bringing them down to a value of 0.1 allows them to have some impact on the neighbourhood but not
#to entirely influence it
    elif d[idx] > 0.1:
        d[idx] = 0.1

#Finally convert all values to their absolutes. This will give places which have a low current correlation with the coffee shops more importance and diversify the dummy neighbourhood for matching against currently existing neighbourhoods
    elif d[idx] < 0:
        d[idx] = np.absolute(d[idx])

#Extract the column names from the toronto correlation to ensure the values are matched correctly
e = list(toronto_correlation)

#Now create a new dataframe from the values which can be appended to the end of the current dataframe that will be used for KNN
df = pd.DataFrame(d)
df = df.transpose()
df.columns = e
df.insert(loc = 0, column = 'Neighbourhood', value='Dummy')



Now that the neighbourhood has been created, compare the distance from this neighbourhood to all others to get the nearest neighbours


In [28]:
dist_neighbours = toronto_grouped.copy()
#dist_neighbours['dist'] = np.nan
dist_neighbours.drop(['Neighbourhood'], axis=1, inplace = True)
df.drop(['Neighbourhood'], axis=1, inplace=True)
#dist(x, y) = sqrt(dot(x, x) - 2 * dot(x, y) + dot(y, y))

from sklearn.metrics.pairwise import euclidean_distances
#dist_neighbours

df

,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0.036473,0.004574,0.004574,0.103727,0.087178,0.087178,0.087178,0.087178,0.087178,0.064398,0.063727,0.076599,0.087178,0.1,0.1,0.0

In [29]:
distances = []

b = euclidean_distances(df, dist_neighbours)

b
dist_neighbours.insert(loc = 0, column = 'Neighbourhood', value = toronto_grouped['Neighbourhood'])
df.insert(loc = 0, column = 'Neighbourhood', value='Dummy')
c = b[0]
c = c.tolist()
c
dist_neighbours['Distance'] = c


Now that we have the distance between the dummy, correlation based neighbourhood and the real neighbourhoods of Toronto we need to find the ten best neighbourhoods in which to open the coffee shop. First, remind ourselves of what neighbourhoods are oversaturated

In [30]:
top_two_venues

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Thai Restaurant,Bar,Steakhouse,Hotel,Restaurant,Burger Joint,Bakery,Sushi Restaurant,American Restaurant
5,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Sandwich Place,Supermarket,Restaurant,Deli / Bodega,Fried Chicken Joint,Frozen Yogurt Shop,Bank,Diner,Bridal Shop
7,"Bedford Park, Lawrence Manor East",Coffee Shop,Italian Restaurant,Fast Food Restaurant,Pharmacy,Sandwich Place,Restaurant,Butcher,Pub,Pizza Place,Comfort Food Restaurant
8,Berczy Park,Coffee Shop,Cocktail Bar,Steakhouse,Cheese Shop,Restaurant,Farmers Market,Italian Restaurant,Seafood Restaurant,Bakery,Beer Bar
11,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Breakfast Spot,Pet Store,Italian Restaurant,Stadium,Caribbean Restaurant,Burrito Place,Furniture / Home Store,Climbing Gym,Bar
15,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Pub,Bakery,Pizza Place,Italian Restaurant,Japanese Restaurant,Liquor Store,Deli / Bodega,Bank
17,Canada Post Gateway Processing Centre,Coffee Shop,Hotel,American Restaurant,Burrito Place,Sandwich Place,Middle Eastern Restaurant,Fried Chicken Joint,Mediterranean Restaurant,Gym / Fitness Center,Dumpling Restaurant
19,Central Bay Street,Coffee Shop,Italian Restaurant,Bubble Tea Shop,Burger Joint,Bar,Chinese Restaurant,Thai Restaurant,Salad Place,Spa,Sushi Restaurant
20,"Chinatown, Grange Park, Kensington Market",Coffee Shop,Bar,Vegetarian / Vegan Restaurant,Bakery,Chinese Restaurant,Mexican Restaurant,Arts & Crafts Store,Vietnamese Restaurant,Caribbean Restaurant,Comfort Food Restaurant
21,Christie,Coffee Shop,Grocery Store,Park,Nightclub,Convenience Store,Diner,Restaurant,Italian Restaurant,Baby Store,Eastern European Restaurant


Now extract the index values and delete them from the distance neighbours, as they are not viable due to their oversaturation

In [31]:
idx_list = top_two_venues.index.values.tolist()
dist_neighbours.drop(idx_list, axis=0, inplace=True)
dist_neighbours.head()

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Distance
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

Now sort the distance dataframe by ascending order to get the closest neighbourhoods to the ideal, 'dummy' neighbourhood


In [32]:
dist_neighbours.sort_values(by ='Distance', ascending = True, inplace=True )
suggested_neighbourhoods = dist_neighbours[0:10]
suggested_neighbourhoods

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Distance
85,"The Beaches West, India Bazaar",0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000

Now remove all of these neighbourhoods from our correlation based matrix of distances to find the ten best neighbourhoods. Get a list of the indexes of the fixed list, then remove them from the second dataframe?


Finally, using foliage, we visualise where these neighbourhoods are in toronto, and check our findings by printing their top ten venues


In [33]:
idx_list = suggested_neighbourhoods.index.values.tolist()
final_suggestions = top_venues.loc[idx_list]
final_suggestions

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
85,"The Beaches West, India Bazaar",Park,Sushi Restaurant,Brewery,Sandwich Place,Burger Joint,Burrito Place,Pub,Coffee Shop,Pet Store,Movie Theater
42,"Fairview, Henry Farm, Oriole",Clothing Store,Fast Food Restaurant,Coffee Shop,Asian Restaurant,Bakery,Electronics Store,Toy / Game Store,Yoga Studio,Juice Bar,Boutique
23,"Clairlea, Golden Mile, Oakridge",Bus Line,Bakery,Bus Station,Metro Station,Soccer Field,Fast Food Restaurant,Park,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
12,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Garden Center,Smoke Shop,Farmers Market,Light Rail Station,Spa,Fast Food Restaurant,Brewery,Burrito Place,Restaurant
46,Glencairn,Park,Italian Restaurant,Japanese Restaurant,Bakery,Arcade,Pub,Yoga Studio,Donut Shop,Dog Run,Doner Restaurant
14,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Terminal,Airport Service,Harbor / Marina,Boat or Ferry,Sculpture Garden,Plane,Boutique,Airport Gate,Airport
62,Lawrence Park,Park,Dim Sum Restaurant,Swim School,Bus Line,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Event Space,Drugstore
40,East Toronto,Metro Station,Park,Coffee Shop,Convenience Store,Yoga Studio,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
44,"Flemingdon Park, Don Mills South",Beer Store,Gym,Asian Restaurant,Coffee Shop,Bike Shop,Dim Sum Restaurant,Japanese Restaurant,Italian Restaurant,Discount Store,Sporting Goods Shop
16,Caledonia-Fairbanks,Park,Fast Food Restaurant,Market,Pharmacy,Women's Store,Gym,Department Store,Ethiopian Restaurant,Empanada Restaurant,Electronics Store


In [34]:
#Extract a list of the neighbourhood names from the above, copy their details to a smaller dataframe then draw the map
final_neighs = []

full_data.set_index('Neighbourhood', drop=True,inplace=True)
for row in final_suggestions.itertuples():
    n = row.Neighbourhood
    final_neighs.append(n)
    
part_data = full_data.loc[final_neighs]

In [35]:
part_data.reset_index(inplace = True)

In [36]:
# create map of Toronto using latitude and longitude values
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(part_data['Latitude'], part_data['Longitude'], part_data['Borough'], part_data['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map